# Visualizing polarization states on the Poincaré Sphere

This is an vizualization tool commonly used to represent polarization states. It has not been broadly adopted in the radioglaciology literature, but in our review we present it in Appendix C as an alternative. Here, we reproduce the figures shown there as well as a few others.

Poincaré, H. (1892). Théorie mathématique de la lumiére, vol. 2 (georges carré, paris). MI MISHCHENKO AND LD TRAVIS, 44.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

from effmed.lib.matrix_model import effective_medium
from impdar.lib.ApresData.load_quadpol import load_quadpol_fujita

%matplotlib inline

In [ ]:
def model(zs,dzs,chis,
          psis=0.001,thetas=0.,
          gammas=None,
          fc = 300e6, Temp = 253., epsr = 3.15):
    # instantiate the model
    em = effective_medium()
    # system properties
    em.system_setup(fc)
    # material properties
    em.epsr = epsr
    em.ice_properties(T=Temp,epsr=epsr,chi=chis[0])
    # solve the model with a unique case if there is only one layer
    if len(chis) == 1:
        em.solve(zs,dzs,thetas,psis,chis[0],gammas=gammas)
    else:
        em.solve(zs,dzs,thetas,psis,chis,gammas=gammas)
    return em

def get_poincare_sphere(em):
    # load the model output as an ImpDAR object
    dat = load_quadpol_fujita(em)
    # rotate through azimuths
    dat.rotational_transform(n_thetas=401)

    # calculate ellipse phase delay
    ϕHH = np.angle(dat.HH)
    ϕHV = np.angle(dat.HV)
    ϕ = ϕHH-ϕHV
    ϕ[ϕ<-np.pi] += 2.*np.pi
    ϕ[ϕ>np.pi] -= 2.*np.pi

    # calculate ellipse rotation
    E0x = abs(dat.HH)
    E0y = abs(dat.HV)
    γ = np.arctan(E0y/E0x)

    # get coordinates on the sphere
    S1 = E0x**2. - E0y**2.
    S2 = 2.*E0x*E0y*np.cos(ϕ)
    S3 = 2.*E0x*E0y*np.sin(ϕ)
    
    return S1,S2,S3

In [ ]:
### Create a wireframe grid and plot it as a reference 

# mesh the grid
u, v = np.mgrid[0:2*np.pi+.01:np.pi/10, 0:np.pi:25j]
x = np.cos(u)*np.sin(v)
y = np.sin(u)*np.sin(v)
z = np.cos(v)

# plot sphere
plt.figure()
ax = plt.subplot(111,projection='3d')
ax.plot_wireframe(y, z, x, color="k", alpha=0.1)
plt.axis('equal')
plt.axis('off')

# find and plot the linear polarization states
N = 100
ts = np.linspace(0,2*np.pi,N)
xx = np.cos(ts)
yy = np.sin(ts)
zz = np.zeros(N)
plt.plot(xx,yy,zz,c='grey')
plt.plot([1,-1,0,0],[0,0,-1,1],[0,0,0,0],'k.')

In [ ]:
### Three representative model scenarios 

# geometry for all
H = 1000.
zs = np.arange(10,H)

# birefringence no anisotropic scattering
chis = np.array([[.1,0.0,0.9]])
em1 = model(zs,H,chis)

# anisotropic scattering no birefringence 
chis = np.array([[.333,.333,.333]])
gammas = np.array([1,.2])
em2 = model(zs,H,chis,gammas=gammas)

# both birefringence and anisotropic scattering
chis = np.array([[.35,.15,0.5]])
gammas = np.array([1,.2])
em3 = model(zs,H,chis,gammas=gammas)

In [ ]:
### Plot all three model scenarios calculated in the previous cell onto the Poincare sphere

plt.figure(figsize=(12,4))
ax1 = plt.subplot(131,projection='3d')
ax2 = plt.subplot(132,projection='3d')
ax3 = plt.subplot(133,projection='3d')

ems = [em1,em2,em3]
axs = [ax1,ax2,ax3]

for i in range(3):
    em = ems[i]
    ax = axs[i]
    ax.plot_wireframe(y, z, x, color="k", alpha=0.025)
    S1, S2, S3 = get_poincare_sphere(em)
    if i == 0:
        for pi in range(0,len(S1[0])//2+2,20):
            ax.plot(S2[:,pi],-S1[:,pi],S3[:,pi],lw=1,c=cm.twilight_shifted(pi/200))
    elif i == 1:
        for pi in range(0,len(S1[0]),20):
            ax.plot(S2[0,pi],-S1[0,pi],S3[0,pi],'.',ms=3.5,c=cm.twilight_shifted(pi/400))
    elif i == 2:
        for pi in range(len(S1[0])-1,-1,-20):
            ax.plot(S2[:,pi],-S1[:,pi],S3[:,pi],lw=1,c=cm.twilight_shifted(pi/400))
    ax.axis('equal')
    ax.axis('off')

In [ ]:
### A fourth model scenario, now with an azimuthal rotation at some depth

# geometry needs to be discretized now
layer_dz = 2.
zs = np.arange(1,H,layer_dz)
dzs = layer_dz*np.ones(len(zs))

# instantaneous rotation by 0.1*pi at 300 m depth
psis = np.zeros_like(zs)
psis[zs>300] = .1*np.pi

# still no oblique propagation (theta=0) and COF eigenvalues are same as in scenario #3
thetas = np.zeros(len(zs))
chis = np.array([[.35,.15,0.5]])
chis = np.tile(chis,(len(zs),1))

# run model
em4 = model(zs,dzs,chis,psis,thetas)

In [ ]:
### Again, plot on the poincare sphere, now choosing specific azimuths in each subpanel
# colored lines are before rotation and gray after

S1, S2, S3 = get_poincare_sphere(em4)

plt.figure(figsize=(12,4))
ax1 = plt.subplot(131,projection='3d')
ax2 = plt.subplot(132,projection='3d')
ax3 = plt.subplot(133,projection='3d')
axs = [ax1,ax2,ax3]

tis = [70,110,130]

for i in range(3):
    ax = axs[i]
    ti = tis[i]
    
    ax.plot(S2[:150,ti],-S1[:150,ti],S3[:150,ti],lw=2,c=cm.twilight_shifted(ti/200))
    ax.plot(S2[150:,ti],-S1[150:,ti],S3[150:,ti],lw=2,c='grey')
    ax.plot(S2[150,ti],-S1[150,ti],S3[150,ti],'.',ms=10,mew=2,mfc='w',c='grey')

    ax.plot_wireframe(y, z, x, color="k", alpha=0.025)
    ax.axis('equal')
    ax.axis('off')